In [1]:
%matplotlib inline

#import theano
import time
import deap
from itertools import izip
import random
from autograd import grad
import autograd.numpy as np
import autograd.numpy.linalg as LA
binarylist = [1, 0]
xorinputs = []
xoroutputs = []
for x in binarylist:
    for y in binarylist:
        xorinputs.append(np.array([x,y]))
        xoroutputs.append(np.array([x^y]))
        

In [2]:
import mnist

#format is (label as int, 2D image array)
TrainingSet = list(mnist.read(dataset = "training")) 
TestingSet = list(mnist.read(dataset = "testing"))
TrainingSet[0]

(5, array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   3,
          18,  18,  18, 126, 136, 175,  26, 166, 255, 247, 127,   0,   0,
         

In [3]:
# def labelToArray(x):
#     blank = [0] * 10
#     blank[x] = 1
#     return np.array(blank)
# TrainingSetList = list(TrainingSet)
# TestSetList = list(TestingSet)
import csv
from sklearn import preprocessing

trainingFilePath = "numerai_training_data.csv"
tournamentFilePath = "numerai_tournament_data.csv"


def readTraining(splitforTesting = True):
    trainingFile = open(trainingFilePath, 'rb')
    trainingFeatures = []
    trainingLabels = []

    rownum = 0
    reader = csv.reader(trainingFile)
    for row in reader:
        # Save header row.
        if rownum == 0:
            header = row
        else:
            features = row[:len(row) - 1 ]
            trainingFeatures.append(np.array(features).astype('float32'))
            trainingLabels.append(np.array([row[-1]]).astype('float32'))

                
        rownum += 1
    
    for i in range(len(trainingLabels)):
        if trainingLabels[i] == np.array([1.0]).astype('float32'):
            trainingLabels[i] = np.array([0.0,1.0]).astype('float32')
        else:
            trainingLabels[i] = np.array([1.0,0.0]).astype('float32')
    
    trainingFeatures = preprocessing.scale(np.array(trainingFeatures))
    trainingFeatures = [np.array(x).astype('float32') for x in trainingFeatures.tolist()]
    if splitforTesting:
        testingFeatures = trainingFeatures[len(trainingFeatures) - 10000 : len(trainingFeatures)]
        testingLabels = trainingLabels[len(trainingLabels) - 10000 : len(trainingLabels)]

        trainingFeatures = trainingFeatures[0:len(trainingFeatures) - 10000]
        trainingLabels = trainingLabels[0:len(trainingLabels) - 10000]
        return trainingFeatures, trainingLabels, testingFeatures, testingLabels

    else:
        return trainingFeatures, trainingLabels

trainingImages, trainingLabels, testImages, testLabels = readTraining(splitforTesting = True)
# trainingLabels = [labelToArray(a) for a,b in TrainingSetList]
# trainingImages = [np.ravel(b) for (a,b) in TrainingSetList]
# testLabels = [labelToArray(a) for a,b in TestSetList]
# testImages = [np.ravel(b) for a,b in TestSetList]
len(trainingLabels)
print trainingImages[0].shape
print trainingLabels[0]

(21,)
[ 0.  1.]


/usr/lib64/python2.7/site-packages/sklearn/preprocessing/data.py:167: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
/usr/lib64/python2.7/site-packages/sklearn/preprocessing/data.py:184: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "


In [4]:
from autograd import grad
def error((weights, biases), inputvec, outputvec):
    return LA.norm(outputvec - feedforward((weights, biases, inputvec)))
errorgrad = grad(error)

def feedforward((weights, biases, inputvecx)):
    for layer, bias in izip(weights, biases):
        inputvecx = np.tanh(inputvecx)
        inputvecx = np.dot(inputvecx, layer) + bias
    return inputvecx

def foo(x):
    if x > 0.5:
        return 1.0
    return 0.0
relu = np.vectorize(lambda x: x * (x> 0))
binarize = np.vectorize(foo)
        

class classicneuralnet:
    def __init__(self, inputsize, hiddenlayersizes, outputsize):
        self.weights = []
        self.biases = []
        self.hiddenlayers = len(hiddenlayersizes)
        self.totalsize = 0
        
        prevsize = inputsize
        hiddenlayersizes.append(outputsize)
        
        for size in hiddenlayersizes:
            self.weights.append(0.05 * np.random.random((prevsize,size)) - 0.05) # used to be prevsize + 1 with no biases
            self.biases.append(0.1* np.random.random((1, size)))
            self.totalsize += (prevsize + 1) * size
            prevsize = size
               
    def feedforward(self, weights, biases, inputvec):
        for layer, bias in izip(weights, biases):
            inputvec = np.tanh(inputvec)
            inputvec = np.dot(inputvec, layer) + bias
        return inputvec
    
    def test(self, inputs, outputs):
        errors = [outp * np.log(feedforward((self.weights, self.biases, inp))) for inp,outp in izip(inputs,outputs)]
        return sum(errors)/len(inputs) * 100.0
        
        
    
    def train(self, inputlist, outputlist, alpha, batchsize, epochs, testinputlist = [], testoutputlist = [], verbose = False):
        inlen = len(inputlist)
        outlen = len(outputlist)
        global error_grad
        if inlen != outlen:
            Exception("number of input vectors ("+str(inlen)+") not equal not number of ouptut vectors ("+str(outlen)+")")
            
        if alpha == 0.0:
            raise("why is alpha zero?")
            
        if type(batchsize) != int:
            raise("Why is batchsize not an int?")
        
        if batchsize == 0 or batchsize > inlen:
            raise("batchsize of "+str(batchsize)+"is not allowed. Note than inputveclist has length "+str(inlen))
            
        #just some error checks
        
        randindexlist = range(inlen)
        start = time.time()
        avgpercenttesterror = "No test set"
        avgpercenttesterrorlist = []
        
        #using stochastic gradient descent training method
        for epoch in xrange(epochs):
            #random.shuffle(randindexlist)
            #for batch in range(inlen/batchsize):
            
#                 low = batch * batchsize
#                 upper = (batch + 1) * batchsize
#                 if upper > inlen: upper = inlen 
                
#                 total_Egradweights = None
#                 total_Egradbiases = None
                
#                 for i in randindexlist[low:upper]:
            for inp,outp in izip(inputlist,outputlist):
                Egradweights, Egradbiases = errorgrad((self.weights, self.biases),
                                                                            inp, outp) #formerly used indexes from randindexlist
#                     if total_Egradweights:
#                         total_Egradweights = [x + y for x,y in izip(Egradweights,total_Egradweights)]
#                         total_Egradbiases = [x + y for x,y in izip(Egradbiases,total_Egradbiases)]
#                     else:
#                         total_Egradweights = Egradweights
#                         total_Egradbiases = Egradbiases
                
#                 total_Egradweights = [x / (batchsize * 1.0) for x in total_Egradweights]
#                 total_Egradbiases = [x / (batchsize * 1.0) for x in total_Egradbiases]
                
#                 self.weights = [curr-grad*alpha for curr, grad in izip(self.weights, total_Egradweights)]
#                 self.biases = [curr-grad*alpha for curr, grad in izip(self.biases, total_Egradbiases)]
                self.weights = [curr-grad*alpha for curr, grad in izip(self.weights, Egradweights)]
                self.biases = [curr-grad*alpha for curr, grad in izip(self.biases, Egradbiases)]
            
            if len(testinputlist) == len(testoutputlist) and len(testinputlist) != 0:
                avgpercenttesterror = self.test(testinputlist, testoutputlist)
                avgpercenttesterrorlist.append(avgpercenttesterror)
            else:
                print "bad test set"
            
            elapsed = time.time() - start
            start = time.time()
            if verbose:
                print "epoch: " + str(epoch) + "\t percenterror: " + str(avgpercenttesterror) + "% \t time elapsed this epoch: " + str(elapsed) + "s"
            
        return (self.weights, self.biases, avgpercenttesterror, avgpercenttesterrorlist)
       

In [5]:
test = classicneuralnet(21,[200,200], 2)

In [ ]:
test.train(trainingImages, trainingLabels, 0.001, 1, 25, testImages, testLabels, verbose = True )

epoch: 0	 percenterror: [[-46.15840547 -26.14615487]]% 	 time elapsed this epoch: 289.859450102s
epoch: 1	 percenterror: [[         nan -26.24315418]]% 	 time elapsed this epoch: 348.201826096s
epoch: 2	 percenterror: [[ nan  nan]]% 	 time elapsed this epoch: 550.997834921s

In [ ]:
trainingImages

In [ ]:
nist = mnist.read()

In [ ]:
nist.next()

In [ ]:
test.weights[0]

In [ ]:
test.biases

In [ ]:
test.feedforward(test.weights,test.biases, np.array([0,1]))

In [ ]:
np.dot(np.array([1,1]), test.weights[0]) + test.biases[0]

In [ ]:
myerror_grad = grad(error)

In [ ]:
myerror_grad((test.weights, test.biases), np.array([1.0,0.0]), np.array([1.0]))

In [ ]:
LA.norm(np.array([1,1]))

In [ ]:
yodle = np.array([1,2])
np.append(yodle, np.array([3]))

In [ ]:
yodle + np.array([0,0,3])


In [ ]:
def f(x):
    return np.tanh(x)
mygradient = grad(f)

In [ ]:
import matplotlib.pyplot as plt
x = np.linspace(-7,7,100)
plt.plot(x, map(f, x),x, map(mygradient, x))
plt.show()

In [50]:
import os
import json
prefix = "unboxedIndividualReluV4"
directory = "/home/helios/Drive/Skynet"
genSize = 10

fnames = [f for f in os.listdir(directory) if prefix in f]

timesandErrors = []
for fname in fnames:
    with open(fname) as f:
        timesandErrors.append((os.path.getctime(fname), json.loads(f.readline())))

timesandErrors.sort(key=lambda x: x[0])
timesandErrors = map (lambda x : x[1], timesandErrors)

chunked = [timesandErrors[i:i+genSize] for i in xrange(0,len(timesandErrors),genSize)]

chunked = chunked[0:len(chunked)-1]
print len(chunked)
minned = map (lambda l: map(min,l), chunked)
print minned

print "first generation " + str(minned[0])
print "fifth generation " + str(minned[-1])
print minned
averages = map (lambda ls: sum(ls)/genSize, minned)
returned = map(lambda l: str('\n'.join(map(str, l))), minned)
for ret in returned:
    print ret
    print
bestIndividuals = map (lambda ls: min(map(lambda l: min(l), ls)), chunked)
print bestIndividuals

5
[[66.79, 36.666, 89.798, 84.6, 89.798, 88.644, 26.052, 36.658, 90.064, 89.798], [78.746, 26.198, 26.45, 55.928, 32.276, 31.29, 89.65, 28.562, 88.644, 28.218], [23.29, 26.948, 80.938, 88.644, 88.644, 89.798, 88.644, 89.65, 89.65, 23.044], [45.28, 20.102, 37.0, 33.524, 38.97, 30.598, 36.538, 88.644, 34.618, 36.778], [47.61, 19.204, 22.58, 16.552, 47.248, 32.586, 13.58, 53.264, 71.46, 56.824]]
first generation [66.79, 36.666, 89.798, 84.6, 89.798, 88.644, 26.052, 36.658, 90.064, 89.798]
fifth generation [47.61, 19.204, 22.58, 16.552, 47.248, 32.586, 13.58, 53.264, 71.46, 56.824]
[[66.79, 36.666, 89.798, 84.6, 89.798, 88.644, 26.052, 36.658, 90.064, 89.798], [78.746, 26.198, 26.45, 55.928, 32.276, 31.29, 89.65, 28.562, 88.644, 28.218], [23.29, 26.948, 80.938, 88.644, 88.644, 89.798, 88.644, 89.65, 89.65, 23.044], [45.28, 20.102, 37.0, 33.524, 38.97, 30.598, 36.538, 88.644, 34.618, 36.778], [47.61, 19.204, 22.58, 16.552, 47.248, 32.586, 13.58, 53.264, 71.46, 56.824]]
66.79
36.666
89.798
8

In [ ]:
import numpy
x,y=numpy.ogrid[-2:1:10000j,-1.5:1.5:10000j]

print('')
print('Grid set')
print('')

c=x + 1j*y
z=0

for g in range(20):
        print('Iteration number: ',g)
        z=z**2 + c

threshold = 2
mask=numpy.abs(z) < threshold

print('')
print('Plotting using imshow()')
plt.imshow(mask.T,extent=[-2,1,-1.5,1.5])

print('')
print('plotting done')
print('')

plt.gray()

print('')
print('Preparing to render')
print('')

plt.show()

In [ ]:
range(3)

In [ ]:
Out[55][0:1]

In [ ]:
[1,2]-[1,2]

In [ ]:
len(xrange(1000))

In [ ]:
type(3) == int

In [ ]:
[1,2,3] + None

In [ ]:
y += 1
y

In [ ]:
if None:
    print "True"

In [ ]:
shuffle = [1,2,3,4] 
random.shuffle(shuffle)
shuffle

In [ ]:
[1,2,3][:]

In [ ]:
np.array([1,2])

In [ ]:
np.random.random((1,2))

In [ ]:
[1,2] + [3,4]

In [ ]:
[1,2] + None

In [ ]:
hi = [1,2,3]
hi /= 1.5

In [ ]:
3/2

In [6]:
import theano
import theano.tensor as T
import numpy as np

In [ ]:
T.vector(numpy.array([1,2]))

In [11]:
import theano
import theano.tensor as T
x = T.fvector()
y = T.fmatrix()
z = T.dot(x, y)
f = theano.function([x,y],z)
f(np.array([1,2]).astype(theano.config.floatX), np.array([range(3), range(3)]).astype(theano.config.floatX))

array([ 0.,  3.,  6.], dtype=float32)

In [89]:
from random import shuffle
import time
inds = T.imatrix('inds')
x = T.fvector('x')
chosen = x.take(inds).sum(axis = 1)
chosen = T.concatenate([chosen, chosen])
theanof = theano.function([inds,x], chosen)
print theanof( np.array([[1,2],[2,3]]).astype('int32'), np.arange(31).astype(theano.config.floatX))
# y, _ = theano.map(fn = lambda i : x[i], sequences = inds)
# scanf = theano.function([inds,x], y)

# listinds = range(10000000)
# npinds = np.array(listinds).astype('int32')
# npx = np.array(listinds).astype(theano.config.floatX)
# # print type(npinds)
# # print type(npx)
# # mylist = np.array(range(31))
# # print mylist
# # print np.choose(np.array([1,2]), mylist)
# testStart = time.time()
# x = theanof(npinds, npx)
# print "choose did it in: " + str(time.time() - testStart)
# # testStart = time.time()
# # x = scanf(npinds, npx)
# # print "map did it in: " + str(time.time() - testStart)
# testStart = time.time()
# x = npx.take(npinds)
# print "python did in in: " + str(time.time() - testStart)

[ 3.  5.  3.  5.]


In [14]:
[1,"hello"]

[1, 'hello']

In [78]:
from theano import tensor as T
x = T.fvector('x')
inds = T.ivector()
y = T.choose(inds,x)
f = theano.function([x,inds], y)
f(np.array(range(31)).astype(theano.config.floatX), np.array([1]).astype('int32'))

array([ 1.], dtype=float32)

In [23]:
int(2.9)

2

In [8]:
%%bash 
cd ~/Drive/Skynet
ghc -O2 -threaded -optc-O3 -optc-ffast-math -rtsopts -fexcess-precision -fprof-auto -fbreak-on-exception fractalUnboxedReluV2.hs

[1 of 1] Compiling Main             ( fractalUnboxedReluV2.hs, fractalUnboxedReluV2.o )
Linking fractalUnboxedReluV2 ...


In [ ]:
import fulcrum
fulcrum.run()

Using gpu device 0: GeForce GTX 970 (CNMeM is disabled, cuDNN not available)
/usr/lib64/python2.7/site-packages/sklearn/preprocessing/data.py:167: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


loading data into RAM...
reformatting data...
10000
86320
[ 1.55478084 -0.35435715 -1.53061199 -0.94231004  0.37014699 -0.23755288
 -1.1931684   1.4748404  -1.39032066 -1.32913303 -0.83353394 -1.41917324
  1.07797825 -1.02009034  0.6895715   1.13847876 -0.27244335 -0.70867389
  1.54909635  0.38860792 -0.30387601]
[ 0.  1.]
data reformatting complete. Beginning evolution. Training is 86320examples, testing 10000
nnetEvaluator testingims length: 10000
evaluating
running initial singleGenConsolidate


/usr/lib64/python2.7/site-packages/sklearn/preprocessing/data.py:184: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "


starting a singleGenConsolidate
half way done a singleGenConsolidate
finished a singleGenConsolidate
starting to loop
starting a singleGenConsolidate
half way done a singleGenConsolidate
finished a singleGenConsolidate
looped
starting a singleGenConsolidate
half way done a singleGenConsolidate
finished a singleGenConsolidate
looped
starting a singleGenConsolidate
half way done a singleGenConsolidate
finished a singleGenConsolidate
looped
starting a singleGenConsolidate
half way done a singleGenConsolidate
finished a singleGenConsolidate
looped
starting a singleGenConsolidate
half way done a singleGenConsolidate
finished a singleGenConsolidate
looped
starting a singleGenConsolidate
half way done a singleGenConsolidate
finished a singleGenConsolidate
looped
starting a singleGenConsolidate
half way done a singleGenConsolidate
finished a singleGenConsolidate
looped
starting a singleGenConsolidate
half way done a singleGenConsolidate
finished a singleGenConsolidate
looped
starting a singleG

In [1]:
%%bash
ps -aux | grep fractal

helios   28940 99.9 27.6 4825216 4479628 tty1  Rl   12:42  21:03 /home/helios/Drive/Skynet/fractalUnboxedReluV2
helios   29037  0.0  0.0 112648   944 tty1     S    13:04   0:00 grep fractal


In [2]:
%%bash
kill 26985

In [1]:
import pandas as pd
train = pd.read_csv("numerai_training_data.csv")

In [5]:
for i in range(1,22):
    print train.groupby('feature' + str(i))['feature' + str(i)].count()

feature1
0.000000      1
0.000031    102
0.000242     93
0.002356     96
0.002505     96
0.002936     87
0.004300     88
0.005682     91
0.007780     98
0.007780    102
0.007780     88
0.009855     98
0.010086     94
0.010706    103
0.010725     99
0.012213     92
0.012652    111
0.012678     88
0.015628    105
0.015687     89
0.015937     85
0.017520     95
0.017528    101
0.019466     95
0.019469     87
0.021334     98
0.021581    107
0.021675     97
0.024439     98
0.025309     89
           ... 
0.966091     97
0.966925     91
0.967753     96
0.967971    106
0.968112     90
0.970254     85
0.970636     87
0.972762     93
0.973878     94
0.974237     97
0.974831     89
0.975005    101
0.975760     93
0.975761     98
0.975892    102
0.977119    105
0.978595     93
0.981748     87
0.982411     98
0.983594     95
0.983627    104
0.985582     93
0.985625     94
0.989442     97
0.989499     93
0.991919    102
0.997193     92
0.997223     94
0.997515    102
1.000000     95
Name: feature1,

In [ ]:
import fulcrum
fulcrum.run()

loading data into RAM...
reformatting data...
10000
86320
[ 1.55478084 -0.35435715 -1.53061199 -0.94231004  0.37014699 -0.23755288
 -1.1931684   1.4748404  -1.39032066 -1.32913303 -0.83353394 -1.41917324
  1.07797825 -1.02009034  0.6895715   1.13847876 -0.27244335 -0.70867389
  1.54909635  0.38860792 -0.30387601]
[ 0.  1.]
data reformatting complete. Beginning evolution. Training is 86320examples, testing 10000
nnetEvaluator testingims length: 10000
evaluating
running initial singleGenConsolidate
starting a singleGenConsolidate
half way done a singleGenConsolidate
finished a singleGenConsolidate
starting to loop
starting a singleGenConsolidate
half way done a singleGenConsolidate
finished a singleGenConsolidate
looped
branchmultiplier: 33
weight sharing: [False, False]
weights number: 0
consolidations lengths[310, 2193]
average number of connections per neuron by layer: [1.0, 2.235483870967742]
spacial dimension: 2
branching by layer: [15, 18]
beginning training

testLen is [ 1.  0.]
e